In [4]:
import gzip
import json
import pandas as pd
import geopandas as gpd
import os
from tqdm import tqdm

In [5]:
# Répertoires d'entrée et de sortie
input_dir = "data"           # Dossier contenant les fichiers .json.gz téléchargés du cadastre
output_dir = "data_parquet"  # Dossier où seront stockés les fichiers convertis en .parquet

# Crée le dossier de sortie s’il n’existe pas (évite une erreur si le dossier est déjà présent)
os.makedirs(output_dir, exist_ok=True)

# Parcourt tous les fichiers présents dans le répertoire d’entrée
for filename in os.listdir(input_dir):
    
    # Ne traite que les fichiers .json.gz
    if filename.endswith(".json.gz"):
        input_path = os.path.join(input_dir, filename)

        # Ouvre le fichier compressé en mode texte
        with gzip.open(input_path, 'rt', encoding='utf-8') as f:
            data = json.load(f)  # Charge le GeoJSON en mémoire

        # Import de shapely pour reconstruire les géométries
        from shapely.geometry import shape
        import warnings
        
        features_ok = []  # Liste des entités valides (géométrie OK)

        # On parcourt chaque feature du GeoJSON
        for feature in data["features"]:
            try:
                # Transforme la géométrie GeoJSON en objet Shapely
                geom = shape(feature["geometry"]) if feature["geometry"] else None

                # Remplace le champ geometry par l'objet Shapely
                feature["geometry"] = geom

                # Ajoute la feature à la liste si aucun problème rencontré
                features_ok.append(feature)

            except Exception as e:
                # Si une géométrie est invalide, on ignore la feature et on émet un avertissement
                warnings.warn(f"Géométrie invalide ignorée: {e}")
        
        # Construit un GeoDataFrame uniquement avec les géométries valides
        gdf = gpd.GeoDataFrame.from_features(features_ok)

        # Détermine le nom du fichier de sortie (.parquet)
        base_name = filename.replace(".json.gz", ".parquet")
        output_path = os.path.join(output_dir, base_name)

        # Enregistre le GeoDataFrame en .parquet
        gdf.to_parquet(output_path)

        # Message de confirmation
        print(f"✅ Converti : {filename} → {base_name}")


✅ Converti : cadastre-12-batiments.json.gz → cadastre-12-batiments.parquet
